# CEO-Request Challenge

## 1. Summary of Problem Statement

❓ **How could Olist improve its profit** ❓

### P&L Rules

#### Revenues  
**Sales fees:** Olist takes a **10% cut** on the product price (excl. freight) of each order delivered  
**Subscription fees:** Olist charges **80 BRL by month** per seller

#### Costs
**Reputation costs** *estimated* per order with bad reviews (<= 3 stars)  

💡 In the long term, bad customer experience has business implications: low repeat rate, immediate customer support cost, refunds or unfavorable word of mouth communication. We will assume that we have an estimate measure of the monetary cost for each bad review:
```python
# review_score: cost(BRL)
{'1 star': 100
'2 stars': 50
'3 stars': 40
'4 stars': 0
'5 stars': 0}
```

**IT costs:** Olist's **total cumulated IT Costs** scale with the square root of the total number of sellers that has ever join the platform, as well as the square root of the total cumulated number of products that was ever sold.  

$IT\_costs = \alpha * \sqrt{n\_sellers} + \beta * \sqrt{n\_products}$  
Olist's data team gave us the following values for these scaling parameters:
- $\alpha = 3157.27$
- $\beta = 978.23$

💡 Both number of sellers to manage and sales transaction are costly for IT systems.  
💡 Yet square roots reflect scale-effects: IT-system are often more efficient as they grow bigger.  
💡 Alpha > Beta means that "Olist has a lower IT Cost with few sellers selling a lot of products rather than the opposite  
- with **1000 sellers** and a total of **100 products** sold, the total IT cost accumulates to 109,624 BRL
- with **100 sellers** and a total of **1000 products** sold, the total IT cost accumulates to 62,507 BRL

Finally, The IT department also told you that since the birth of the marketplace, cumulated IT costs have amounted to **500,000 BRL**.

### Key Findings, so far

- `wait_time` is the most significant factor behind low review scores  
- `wait_time` is made up of seller's `delay_to_carrier` + `carrier_delivery_time`.  
- The latter being outside of Olist's direct control, improving it is not a quick-win recommendation
- On the contrary, a better selection of `sellers` can positively impact the `delay_to_carrier` and reduce the number of bad `review_scores` on Olist.
- Comments of the bad reviews showed that some were linked to the seller or to the product itself.

💡 We recommend you to start with the the guided seller analysis in part 2 below  
💪 But feel free to investigate into other hypothesis instead with part 3

## 2. Should Olist remove under-performing sellers from its marketplace? 🕵🏻
*(recommended)*

To analyze the impact of removing the worse sellers from Olist's marketplace, we will perform a **what-if analysis**

👉 **What would have happened if Olist had never accepted these sellers in the first place?**  

*(In practice, it's hard to know in advance who is a good seller, but let's start with this approach and iterate later).*

### 2.1 Data Preparation

Compute, for each `seller_id`, and cumulated since the beginning:
- the `revenues` it brings
- the `review_costs` associated with all its bad reviews
- the resulting `profits` (revenues - costs)

👉 Write down a step-by-step strategy to create the DataFrame you need


⚠️ Don't start from scratch, update your existing package 😉

Starting from the `Seller` class of your `olist` package:

Edit the `get_training_data` method so that the DataFrame it returns contains the fields:
- `revenues`: sum of subscription and sales fees revenues
- `cost_of_reviews`: sum of costs associated with bad reviews
- `profits`: `revenues` - `cost_of_reviews`

### 2.2 What-if Analysis

👉 Time to perform the actual analysis, here are our steps:  

1️⃣ Create a method that will help us update the IT Costs after removing sellers along with the products they sold

2️⃣ Sort sellers by increasing profits

3️⃣ Remove sellers one by one, starting from the one with the lowest profit.
- For each number of sellers to remove, compute the financial impact on Olist global profits.

4️⃣ Find an optimal number of sellers to remove that maximizes either Olist's profit margin or profit.

1. Revenues

    1.1. Pegar todas as orders de um seller
    
    1.2. Add todos of prices e pegar 10%
    
    1.3. Numero de meses no Olist * 80
    
    1.4. Somar os dois para obter o revenue do seller
    
    
2. Costs

    2.1. Atualizar get_review_score para ter share_of_n_star
    
    2.2. Multiplicar cada share_of_n_star pelo custo do mesmo
    
    2.3. Soma todos os custos de review por seller
    
  
3. Profit

    3.1. Calcular o profit de cada seller (revenue - cost of review)
    
    3.2. Sort os sellers por melhor para pior
    
    3.3. Somar o profits de todos os sellers
    
    3.4. Calcular o custo de IT
    
    3.5. Função que remove um por um os piores vendedores e os produtos dele e retorna o profit total dos que restam
    
    3.6. Plot da evolução do profit conforme removemos os piores sellers
    
    3.7. Observa o numero optimal de vendedores aka o cume da curva
    

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

In [3]:
from olist.seller import Seller
from olist.data import Olist

sellers = Seller().get_training_data().copy()
orders = Olist().get_data()["order_items"].copy()
products = Olist().get_data()["products"].copy()

In [10]:
def compute_profit(sellers, products):
    sellers["sales_fee"] = [sales * 0.1 for sales in sellers["sales"]]
    sellers["subscription_fee"] = [months * 80 for months in sellers["months_on_olist"]]
    sellers["revenue"] = sellers["sales_fee"] + sellers["subscription_fee"]
    sellers["profit"] = sellers["revenue"] - sellers["reviews_cost"]
    sellers = sellers.sort_values(by="profit", ascending=False)
    
    print(sellers.head(5))

    total_profit = sellers["profit"].sum()

    alpha = 3157.27
    beta = 978.23
    number_of_sellers = sellers["seller_id"].count()
    number_of_products = products["product_id"].count() # sellers["quantity"].sum()

    it_costs = alpha * np.sqrt(number_of_sellers) + beta * np.sqrt(number_of_products)
    
    return {"profit": total_profit, "it_costs": it_costs, "sellers_df": sellers}

In [5]:
def remove_seller_products(seller_id, orders, products):
    seller_orders = orders[orders["seller_id"] == seller_id]
    seller_products = products[~products["product_id"].isin(seller_orders["product_id"])]
    #return seller_products
    
def remove_seller(seller_id, orders, products, sellers):
    remove_seller_products(seller_id, orders, products)
    #return sellers[~sellers["seller_id"] == seller_id]

In [12]:
computation = compute_profit(sellers, products)
sellers = computation["sellers_df"]
print(sellers.head(5))

                          seller_id        seller_city seller_state  \
0  3442f8959a84dea7ee197c632cb2df15           campinas           SP   
1  d1b65fc7debc3361ea86b5f14c68d2e2         mogi guacu           SP   
2  ce3ad9de960102d0677a81f5d0bb7b2d     rio de janeiro           RJ   
3  c0f3eea2e14555b6faeea3dd58c1b1c3          sao paulo           SP   
4  51a04a8a6bdcb23deccc82b0b80742cf  braganca paulista           SP   

   delay_to_carrier  wait_time     date_first_sale      date_last_sale  \
0          0.000000  13.018588 2017-05-05 16:25:11 2017-08-30 12:50:19   
1          0.000000   9.065716 2017-03-29 02:10:34 2018-06-06 20:15:21   
2          0.000000   4.042292 2018-07-30 12:44:49 2018-07-30 12:44:49   
3          0.000000   5.667187 2018-08-03 00:44:08 2018-08-03 00:44:08   
4          3.353727  35.314861 2017-11-14 12:15:25 2017-11-14 12:15:25   

   months_on_olist  n_orders  quantity  quantity_per_order     sales  \
0              4.0         3         3               1.0

## 3. Investigate other Approaches 🕵️
*(optional)*

- Should Olist remove the worst performing products / categories from its marketplace entirely?
- Should Olist remove only consistently underperforming sellers, after it has a honeymoon period of few months?
- Should Olist restrict seller/customer pairs between certain states to avoid delays?
- Should Olist acquire new sellers, with some cost assumptions to be suggested?
- ...


## Your turn!

🧺 Keep this notebook tidy! 

🗣 📊 You will present your insights to your favorite TA at the end of this `Communicate` unit 💪